In [ ]:
#This is the code for diesease pre-diagnosis using the Classifier—chain approach.
#SVM, Logistic Regression, XGBoost and Random forest were avaliable using this code
import numpy as np
import pandas as pd
import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
matplotlib.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']= False
matplotlib.rcParams['font.size']= 11
from pandas import DataFrame
from sklearn import svm
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import RocCurveDisplay, roc_curve, auc, roc_auc_score
from sklearn.metrics import hamming_loss, f1_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_classif,f_classif
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.utils import resample
import warnings

list = []
# This section imports the data. We provide demo data “Demo.csv” for code implementation.
filepath = "C:\\Users\\PS\\Desktop\\Open code\\Demo.csv" #Set the user's own path.
df1=[]
df1 = pd.read_csv(filepath, header=0, encoding="gbk")
X1 = df1.drop(["Pancreas disease", "Biliary tract disease", "Gastroduodenal disease", "Intestinal tract disease", "liver disease", "others"], axis=1)
y1 = df1[["Pancreas disease", "Intestinal tract disease", "liver disease", "Biliary tract disease", "Gastroduodenal disease", "others"]]
y= y1.values

#This section initializes the model. Choose one of the four models(SVM, LR, XGBoost, RF) and ensure consistent model selection throughout.
#SVM
classifier5 = SVC(C=1.0, gamma='auto', kernel='linear', class_weight='balanced', tol=1e-2, probability=True)

#Logistic Regression
#classifier1 = LogisticRegression(penalty='l2', dual=False, tol=1e-3, C=1.0, fit_intercept=True,
                                #intercept_scaling=1, class_weight='balanced', random_state=None,
                                #max_iter=100,verbose=0,warm_start=False, n_jobs=-1)
#XGBoost
#classifier4= XGBClassifier(base_score=0.5,booster='gbtree',colsample_bylevel=1,
                            #colsample_bynode=1,colsample_bytree=1,gamma=0.1,
                            #learning_rate=0.3,max_delta_step=0, max_depth=9,
                            #min_child_weight=1,n_estimators=200,n_jobs=-1,
                            #nthread=None,objective='binary:logistic', reg_alpha=0,
                            #reg_lambda=1,scale_pos_weight=1,silent=None,
                            #subsample=1,verbosity=1)

#Random forest
#classifier2 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1)


#This function calculates class weights. This function matches the XGBoost model.
def weight_Class (y):
    # Calculate weight of each subclass.
    counts = sum(y)
    total_samples = len(y)
    weights = total_samples / counts
    
    return weights

#This function performs model training and outputs the optimal model, incorporating 5-fold cross-validation.
def CV_train(classifier, param_grid, X, y):
    #N-fold internal cross-validation
    N = 5
    kf = KFold(n_splits=N, shuffle=True, random_state=3)
    grid_search = GridSearchCV(estimator = classifier, param_grid=param_grid, scoring='roc_auc', cv=kf, n_jobs=-1)
    best_accuracy = 0
    best_clf = None
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]       
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        #Prediction
        pred_train = best_model.predict(X_train)
        pred_test = best_model.predict_proba(X_test)
        #Here the model evaluation criterion is selected. In this example, ROC-AUC is chosen.
        accuracy_val = roc_auc_score(y_test, pred_test[:, 1])
        if accuracy_val > best_accuracy:
           best_accuracy = accuracy_val
           best_clf = best_model
    return best_clf  

# External validation funciton with bootstrap 
def bootstrap_with_roc(classifiers, X, y, n_iterations):

    #Storage space for results
    bootstrap_stats= {
        'recall': [],
        'auc': [],
        'prc': [],
        'accuracy': [],
        'precision': [],  
        'F1': [],
        'han':[]
    }
    bootstrap_stats_class1= {
        'recall': [],
        'specificity': [],
        'auc': [],
        'prc': [],
        'accuracy': [],
        'precision': [],  
        'F1': [] 
    }
    bootstrap_stats_class2={
        'recall': [],
        'specificity': [],
        'auc': [],
        'prc': [],
        'accuracy': [],
        'precision': [],  
        'F1': [] 
    }
    bootstrap_stats_class3= {
        'recall': [],
        'specificity': [],
        'auc': [],
        'prc': [],
        'accuracy': [],
        'precision': [],  
        'F1': [] 
    }
    bootstrap_stats_class4= {
        'recall': [],
        'specificity': [],
        'auc': [],
        'prc': [],
        'accuracy': [],
        'precision': [],  
        'F1': [] 
    }
    bootstrap_stats_class5= {
        'recall': [],
        'specificity': [],
        'auc': [],
        'prc': [],
        'accuracy': [],
        'precision': [],  
        'F1': [] 
    }
    bootstrap_stats_class6= {
        'recall': [],
        'specificity': [],
        'auc': [],
        'prc': [],
        'accuracy': [],
        'precision': [],  
        'F1': [] 
    }
    
    #Transform y into a multi-label dataFrame
    num_samples = X.shape[0]
    y = pd.DataFrame(y, columns=[f'pred_{j}' for j in range(6)])
    num_classifiers = len(classifiers)

    for b in range(n_iterations):
        #resample in bootstrap
        indices_resampled = resample(X.index, n_samples=num_samples, replace=True, random_state=b)
        X_resampled = X.loc[indices_resampled].reset_index(drop=True)
        y_test = y.loc[indices_resampled].reset_index(drop=True)
        y_test_labels= y_test.values
        predictions = np.zeros((num_samples, num_classifiers), dtype=float)  # Floating - point numbers are adopted to maintain precision
        predictions_p = np.zeros((num_samples, num_classifiers), dtype=float)  # Floating - point numbers are adopted to maintain precision  
        for i, (clf, support) in enumerate(classifiers):
            X_selected = X_resampled.iloc[:, support]  # Select the features supported by the current classifier
            if i > 0:
               prev_predictions = predictions[:, :i]  # Retrieve the previous diesease predictions of the previous classifier
               prev_predictions_df = pd.DataFrame(prev_predictions, columns=[f'pred_{j}' for j in range(i)])
               X_selected_T = pd.concat([X_selected, prev_predictions_df], axis=1)
               predictions_b = clf.predict(X_selected_T.values)
               predictions_b_prob = clf.predict_proba(X_selected_T.values) 
            else: 
               predictions_b = clf.predict(X_selected)
               predictions_b_prob = clf.predict_proba(X_selected) 
            # Store the predictions of the current classifier.
            predictions[:, i] += predictions_b
            # Store the prediction probabilities of the current classifier for future use.
            predictions_p[:, i] += predictions_b_prob[:, 1]

        #This section calculates and stores the micro - metrics.
        hamming_loss_val = hamming_loss(y_test_labels, predictions) #hamming_loss
        f1_val = f1_score(y_test_labels, predictions, average='micro') #micro-F1
        precision_val = precision_score(y_test_labels, predictions, average='micro') #Micro-precision
        accuracy_val = accuracy_score(y_test_labels, predictions) # Micro-accuracy
        recall_val = recall_score(y_test_labels, predictions, average='micro') #Mirco-recall
        micro_auc = roc_auc_score(y_test_labels.ravel(), predictions_p.ravel(), average='micro') #Mirco-AUROC
        precisionq, recallq, _ = precision_recall_curve(y_test_labels.ravel(), predictions_p.ravel())
        micro_prc_auc = auc(recallq, precisionq) # 计算 Micro-AUPRC
        bootstrap_stats['recall'].append(recall_val)
        bootstrap_stats['precision'].append(precision_val)
        bootstrap_stats['accuracy'].append(accuracy_val)
        bootstrap_stats['F1'].append(f1_val)
        bootstrap_stats['auc'].append(micro_auc)
        bootstrap_stats['prc'].append(micro_prc_auc)
        bootstrap_stats['han'].append(hamming_loss_val)
        
        # Calculate the multi-label confusion matrix.
        mcm = multilabel_confusion_matrix(y_test_labels, predictions)
        # Compute and store the precision, recall, and F1 score for each subclass.
        n_classes = y.shape[1]
        precision = precision_score(y_test_labels, predictions, average=None)
        recall = recall_score(y_test_labels, predictions, average=None)
        f1 = f1_score(y_test_labels, predictions, average=None)
        bootstrap_stats_class1['precision'].append(precision[0])
        bootstrap_stats_class2['precision'].append(precision[1])
        bootstrap_stats_class3['precision'].append(precision[2])
        bootstrap_stats_class4['precision'].append(precision[3])
        bootstrap_stats_class5['precision'].append(precision[4])
        bootstrap_stats_class6['precision'].append(precision[5])
        bootstrap_stats_class1['recall'].append(recall[0])
        bootstrap_stats_class2['recall'].append(recall[1])
        bootstrap_stats_class3['recall'].append(recall[2])
        bootstrap_stats_class4['recall'].append(recall[3])
        bootstrap_stats_class5['recall'].append(recall[4])
        bootstrap_stats_class6['recall'].append(recall[5])
        bootstrap_stats_class1['F1'].append(f1[0])
        bootstrap_stats_class2['F1'].append(f1[1])
        bootstrap_stats_class3['F1'].append(f1[2])
        bootstrap_stats_class4['F1'].append(f1[3])
        bootstrap_stats_class5['F1'].append(f1[4])
        bootstrap_stats_class6['F1'].append(f1[5])

        #Calculate and store the accuracy for each subclass.
        accuracies = []
        for i in range(n_classes):
            # Extract the true and predicted labels for the i-th class.
            y_true_class = y_test_labels[:, i]
            y_pred_class = predictions[:, i]  # Use binary labels for the i-th class.
            # Calculate the accuracy for the i-th class.
            accuracy_class = accuracy_score(y_test_labels[:, i], y_pred_class)
            if i==0:
               bootstrap_stats_class1['accuracy'].append(accuracy_class)
            elif i==1:
               bootstrap_stats_class2['accuracy'].append(accuracy_class)
            elif i==2:
               bootstrap_stats_class3['accuracy'].append(accuracy_class)
            elif i==3:
               bootstrap_stats_class4['accuracy'].append(accuracy_class)
            elif i==4:
               bootstrap_stats_class5['accuracy'].append(accuracy_class)
            elif i==5:
               bootstrap_stats_class6['accuracy'].append(accuracy_class)

        # Calculate and store the specificity for each class
        specificities = []
        for i in range(n_classes):
            #True Negatives = Sum of diagonal elements - True Positives of the current class.
            true_negatives = np.sum(mcm[:, 0, 0]) - mcm[i, 0, 0]
            # False Positives = Sum of the elements in current row - True Positives.
            false_positives = np.sum(mcm[i, 0, 1])
            # Calculate specificities
            specificity = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0
            if i==0:
               bootstrap_stats_class1['specificity'].append(specificity)
            elif i==1:
               bootstrap_stats_class2['specificity'].append(specificity)
            elif i==2:
               bootstrap_stats_class3['specificity'].append(specificity)
            elif i==3:
               bootstrap_stats_class4['specificity'].append(specificity)
            elif i==4:
               bootstrap_stats_class5['specificity'].append(specificity)
            elif i==5:
               bootstrap_stats_class6['specificity'].append(specificity)

        #Calculate and store AUROC for each class
        fprs = dict()
        tprs = dict()
        roc_aucs = dict()
        for i in range(n_classes):
            fprs[i], tprs[i], _ = roc_curve(y_test_labels[:, i], predictions_p[:, i])
            roc_aucs[i] = auc(fprs[i], tprs[i])
            if i==0:
               bootstrap_stats_class1['auc'].append(roc_aucs[0])
            elif i==1:
               bootstrap_stats_class2['auc'].append(roc_aucs[1])
            elif i==2:
               bootstrap_stats_class3['auc'].append(roc_aucs[2])
            elif i==3:
               bootstrap_stats_class4['auc'].append(roc_aucs[3])
            elif i==4:
               bootstrap_stats_class5['auc'].append(roc_aucs[4])
            elif i==5:
               bootstrap_stats_class6['auc'].append(roc_aucs[5])

        #Calculate and store AUPRC for each class
        precisions = dict()
        recalls = dict()
        pr_aucs = dict()
        for i in range(n_classes):
            precisions[i], recalls[i], _ = precision_recall_curve(y_test_labels[:, i], predictions_p[:, i])
            pr_aucs[i] = auc(recalls[i], precisions[i])
            if i==0:
               bootstrap_stats_class1['prc'].append(pr_aucs[0])
            elif i==1:
               bootstrap_stats_class2['prc'].append(pr_aucs[1])
            elif i==2:
               bootstrap_stats_class3['prc'].append(pr_aucs[2])
            elif i==3:
               bootstrap_stats_class4['prc'].append(pr_aucs[3])
            elif i==4:
               bootstrap_stats_class5['prc'].append(pr_aucs[4])
            elif i==5:
               bootstrap_stats_class6['prc'].append(pr_aucs[5])

    return bootstrap_stats, bootstrap_stats_class1, bootstrap_stats_class2, bootstrap_stats_class3, bootstrap_stats_class4, bootstrap_stats_class5, bootstrap_stats_class6


#Automatically identify continuous and binary variables.
continuous_vars = []
binary_vars = []
for col in X1.columns:
# Automatically identify binary variables by iterating through each column in the dataset. 
#If a column is numeric, has exactly two unique values, and those values are 0 and 1, it is considered a binary variable.
    if X1[col].dtype.kind in 'biufc' and X1[col].nunique() == 2 and set(X1[col].unique()) == {0, 1}:
       binary_vars.append(col)
    # Otherwise, it is considered a continuous variable.
    else:
       continuous_vars.append(col)
    # Define variable groups.
    groups = {
        'Continuous': continuous_vars,
         'Binary': binary_vars
    }

# Apply logarithmic transformation and standardization to continuous variables.
log_X1 = X1.copy()
log_X1[continuous_vars] = np.log1p(X1[continuous_vars])  #Apply logarithmic transformation to continuous variables.
scaler = MinMaxScaler()
X_scaled = log_X1.copy()
X_scaled[continuous_vars] = scaler.fit_transform(X_scaled[continuous_vars])  #Apply standardization to continuous variables.

# Perform train - test split
X_scaled = X_scaled.values
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=5)

# Initialize a list of classifier chains
classifiers = []
# Create a binary classifier for each label
n_classes = y.shape[1]
for i in range(n_classes):
    best_clf = None
    best_accuracy = 0
    #Create a boolean mask to select samples where the current class is active.
    y_train_i = np.where(y_train[:, i] == 1, 1, 0)  #Create a boolean mask where samples are labeled 1 if they belong to class i, and 0 otherwise.
    if np.sum(y_train_i == 1) > 1:  #Ensure there are sufficient positive samples for the model
        #Use RFECV for feature selection
        rfecv = RFECV(estimator=classifier5, step=1, cv=5, scoring='accuracy')
        X_train_1 = pd.DataFrame(X_train, columns=X1.columns)
        rfecv.fit(X_train_1, y_train_i.ravel())
        support = rfecv.support_
        #The parameter scale_weight is tailored for use with XGBoost.
        scale_weight= weight_Class (y_train_i)

        #Reinitialize the classifier here and set up the parameter groups for Gridsearch, ensuring consistency in selection across the four classifiers.
        #SVM
        classifier5 =SVC(C=1.0, gamma='auto', class_weight='balanced', kernel='linear', tol=1e-2, probability=True)
        param_grid1 = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'],  
                       'tol': [1e-3, 1e-2, 1e-1] }

        #LR
        #lassifier1= LogisticRegression(dual=False, fit_intercept=True, intercept_scaling=1, class_weight='balanced', random_state=None,
                                        #verbose=0, warm_start=False, n_jobs=-1)
        #param_grid1 = {'penalty': ['l2'],  'C': [0.5, 1, 1.5, 2],   
                       #'solver': ['newton-cg', 'lbfgs', 'liblinear'],  'max_iter': [1000, 5000],  'tol': [1e-3, 1e-2]}

        #RF
        #classifier2 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1)
        #param_grid1 = {'n_estimators': [100, 200, 300],  'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 
                      #'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}

        #XGBoost
        #classifier4 = XGBClassifier(objective='binary:logistic', scale_pos_weight = scale_weight, n_estimators=100, n_jobs=-1, verbosity=1)
        #param_grid1 = {'max_depth': [6, 9], 'min_child_weight': [3, 5], 'learning_rate': [0.01, 0.1], 'subsample': [0.8, 1.0],
                       #'colsample_bytree': [0.6, 0.8, 1.0], 'gamma': [0, 0.1, 0.2], 'reg_alpha': [0, 0.1, 1.0], 'reg_lambda': [1, 1.1, 10.0]}
        
        y_train_1 = pd.DataFrame(y_train, columns = y1.columns)
        if i > 0:
            X_train_2 = pd.concat([X_train_1.iloc[:, support], y_train_1.iloc[:, :i]], axis=1)
            best_clf = CV_train(classifier5, param_grid1, X_train_2, y_train_i.ravel())
        else:
            best_clf = CV_train(classifier5, param_grid1, X_train_1.iloc[:, support], y_train_i.ravel())

    classifiers.append((best_clf, support))
    
# Use the prediction function for validation with bootstrap
warnings.filterwarnings('ignore', category=UserWarning)
n_iterations=600
bootstrap_stats, bootstrap_stats_class1, bootstrap_stats_class2, bootstrap_stats_class3, bootstrap_stats_class4, bootstrap_stats_class5, bootstrap_stats_class6 = bootstrap_with_roc(classifiers, pd.DataFrame(X_test, columns=X1.columns), y_test, n_iterations)
bootstrap_stat, bootstrap_stat_class1, bootstrap_stat_class2, bootstrap_stat_class3, bootstrap_stat_class4, bootstrap_stat_class5, bootstrap_stat_class6 = bootstrap_with_roc(classifiers, pd.DataFrame(X_train, columns=X1.columns), y_train, n_iterations)

# Print the metric results
print(f"micro- Ave-hamming distance in external validation: {np.mean(bootstrap_stats['han']):.4f}", f"micro- Std-hamming distance in external validation: {np.std(bootstrap_stats['han']):.4f}")
print(f"micro- Ave-sensitivity in external validation: {np.mean(bootstrap_stats['recall']):.4f}", f"micro- Std-sensitivity in external validation: {np.std(bootstrap_stats['recall']):.4f}")
print(f"micro- Ave-AUROC in external validation: {np.mean(bootstrap_stats['auc']):.4f}", f"micro- Std-AUROC in external validation: {np.std(bootstrap_stats['auc']):.4f}")
print(f"micro- Ave-precision in external validation: {np.mean(bootstrap_stats['precision']):.4f}", f"micro- Std-precision in external validation: {np.std(bootstrap_stats['precision']):.4f}")
print(f"subset-Ave-accyracy in external validation: {np.mean(bootstrap_stats['accuracy']):.4f}", f"subset- Std-accuracy in external validation: {np.std(bootstrap_stats['accuracy']):.4f}")
print(f"micro- Ave-F1 in external validation: {np.mean(bootstrap_stats['F1']):.4f}", f"micro- Std-F1 in external validation: {np.std(bootstrap_stats['F1']):.4f}")
print(f"micro- Ave-AUPRC in external validation: {np.mean(bootstrap_stats['prc']):.4f}", f"micro- Std-F1 in external validation: {np.std(bootstrap_stats['prc']):.4f}")

print(f"micro- Ave-hamming distance in internal validation: {np.mean(bootstrap_stat['han']):.4f}", f"micro- Std-hamming distance in internal validation: {np.std(bootstrap_stat['han']):.4f}")
print(f"micro- Ave-sensitivity in internal validation: {np.mean(bootstrap_stat['recall']):.4f}", f"micro- Std-sensitivity in internal validation: {np.std(bootstrap_stat['recall']):.4f}")
print(f"micro- Ave-AUROC in internal validation: {np.mean(bootstrap_stat['auc']):.4f}", f"micro- Std-AUROC in internal cross-validation: {np.std(bootstrap_stat['auc']):.4f}")
print(f"micro- Ave-precision in internal validation: {np.mean(bootstrap_stat['precision']):.4f}", f"micro- Std-precision in internal validation:  {np.std(bootstrap_stat['precision']):.4f}")
print(f"subset-Ave-accyracy in internal validation: {np.mean(bootstrap_stat['accuracy']):.4f}", f"subset- Std-accuracy in internal validation: {np.std(bootstrap_stat['accuracy']):.4f}")
print(f"micro- Ave-F1 in internal validation: {np.mean(bootstrap_stat['F1']):.4f}", f"micro- Std-F1 in internal validation: {np.std(bootstrap_stat['F1']):.4f}")
print(f"micro- Ave-AUPRC in internal validation: {np.mean(bootstrap_stat['prc']):.4f}", f"micro- Std-F1 in internal validation: {np.std(bootstrap_stat['prc']):.4f}")

print(f"subclass1- Ave-specificity in external validation: {np.mean(bootstrap_stats_class1['specificity']):.4f}", f"subclass1- Std-specificity in external validation: {np.std(bootstrap_stats_class1['specificity']):.4f}")
print(f"subclass1- Ave-sensitivity in external validation: {np.mean(bootstrap_stats_class1['recall']):.4f}", f"subclass1- Std-sensitivity in external validation: {np.std(bootstrap_stats_class1['recall']):.4f}")
print(f"subclass1- Ave-AUROC in external validation: {np.mean(bootstrap_stats_class1['auc']):.4f}", f"subclass1- Std-AUROC in external validation: {np.std(bootstrap_stats_class1['auc']):.4f}")
print(f"subclass1- Ave-precision in external validation: {np.mean(bootstrap_stats_class1['precision']):.4f}", f"subclass1- Std-precision in external validation: {np.std(bootstrap_stats_class1['precision']):.4f}")
print(f"subclass1- Ave-accyracy in external validation: {np.mean(bootstrap_stats_class1['accuracy']):.4f}", f"subclass1- Std-accuracy in external validation: {np.std(bootstrap_stats_class1['accuracy']):.4f}")
print(f"subclass1- Ave-F1 in external validation: {np.mean(bootstrap_stats_class1['F1']):.4f}", f"subclass1- Std-F1 in external validation: {np.std(bootstrap_stats_class1['F1']):.4f}")
print(f"subclass1- Ave-AUPRC in external validation: {np.mean(bootstrap_stats_class1['prc']):.4f}", f"subclass1- Std-AUPRC in external validation: {np.std(bootstrap_stats_class1['prc']):.4f}")

print(f"subclass2- Ave-specificity in external validation: {np.mean(bootstrap_stats_class2['specificity']):.4f}", f"subclass2- Std-specificity in external validation: {np.std(bootstrap_stats_class2['specificity']):.4f}")
print(f"subclass2- Ave-sensitivity in external validation: {np.mean(bootstrap_stats_class2['recall']):.4f}", f"subclass2- Std-sensitivity in external validation: {np.std(bootstrap_stats_class2['recall']):.4f}")
print(f"subclass2- Ave-AUROC in external validation: {np.mean(bootstrap_stats_class2['auc']):.4f}", f"subclass2- Std-AUROC in external validation: {np.std(bootstrap_stats_class2['auc']):.4f}")
print(f"subclass2- Ave-precision in external validation: {np.mean(bootstrap_stats_class2['precision']):.4f}", f"subclass2- Std-precision in external validation: {np.std(bootstrap_stats_class2['precision']):.4f}")
print(f"subclass2- Ave-accyracy in external validation: {np.mean(bootstrap_stats_class2['accuracy']):.4f}", f"subclass2- Std-accuracy in external validation: {np.std(bootstrap_stats_class2['accuracy']):.4f}")
print(f"subclass2- Ave-F1 in external validation: {np.mean(bootstrap_stats_class2['F1']):.4f}", f"subclass2- Std-F1 in external validation: {np.std(bootstrap_stats_class2['F1']):.4f}")
print(f"subclass2- Ave-AUPRC in external validation: {np.mean(bootstrap_stats_class2['prc']):.4f}", f"subclass2- Std-AUPRC in external validation: {np.std(bootstrap_stats_class2['prc']):.4f}")

print(f"subclass3- Ave-specificity in external validation: {np.mean(bootstrap_stats_class3['specificity']):.4f}", f"subclass3- Std-specificity in external validation: {np.std(bootstrap_stats_class3['specificity']):.4f}")
print(f"subclass3- Ave-sensitivity in external validation: {np.mean(bootstrap_stats_class3['recall']):.4f}", f"subclass3- Std-sensitivity in external validation: {np.std(bootstrap_stats_class3['recall']):.4f}")
print(f"subclass3- Ave-AUROC in external validation: {np.mean(bootstrap_stats_class3['auc']):.4f}", f"subclass3- Std-AUROC in external validation: {np.std(bootstrap_stats_class3['auc']):.4f}")
print(f"subclass3- Ave-precision in external validation: {np.mean(bootstrap_stats_class3['precision']):.4f}", f"subclass3- Std-precision in external validation: {np.std(bootstrap_stats_class3['precision']):.4f}")
print(f"subclass3- Ave-accyracy in external validation: {np.mean(bootstrap_stats_class3['accuracy']):.4f}", f"subclass3- Std-accuracy in external validation: {np.std(bootstrap_stats_class3['accuracy']):.4f}")
print(f"subclass3- Ave-F1 in external validation: {np.mean(bootstrap_stats_class3['F1']):.4f}", f"subclass3- Std-F1 in external validation: {np.std(bootstrap_stats_class3['F1']):.4f}")
print(f"subclass3- Ave-AUPRC in external validation: {np.mean(bootstrap_stats_class3['prc']):.4f}", f"subclass3- Std-AUPRC in external validation: {np.std(bootstrap_stats_class3['prc']):.4f}")

print(f"subclass4- Ave-specificity in external validation: {np.mean(bootstrap_stats_class4['specificity']):.4f}", f"subclass4- Std-specificity in external validation: {np.std(bootstrap_stats_class4['specificity']):.4f}")
print(f"subclass4- Ave-sensitivity in external validation: {np.mean(bootstrap_stats_class4['recall']):.4f}", f"subclass4- Std-sensitivity in external validation: {np.std(bootstrap_stats_class4['recall']):.4f}")
print(f"subclass4- Ave-AUROC in external validation: {np.mean(bootstrap_stats_class4['auc']):.4f}", f"subclass4- Std-AUROC in external validation: {np.std(bootstrap_stats_class4['auc']):.4f}")
print(f"subclass4- Ave-precision in external validation: {np.mean(bootstrap_stats_class4['precision']):.4f}", f"subclass4- Std-precision in external validation: {np.std(bootstrap_stats_class4['precision']):.4f}")
print(f"subclass4- Ave-accyracy in external validation: {np.mean(bootstrap_stats_class4['accuracy']):.4f}", f"subclass4- Std-accuracy in external validation: {np.std(bootstrap_stats_class4['accuracy']):.4f}")
print(f"subclass4- Ave-F1 in external validation: {np.mean(bootstrap_stats_class4['F1']):.4f}", f"subclass4- Std-F1 in external validation: {np.std(bootstrap_stats_class4['F1']):.4f}")
print(f"subclass4- Ave-AUPRC in external validation: {np.mean(bootstrap_stats_class4['prc']):.4f}", f"subclass4- Std-AUPRC in external validation: {np.std(bootstrap_stats_class4['prc']):.4f}")

print(f"subclass5- Ave-specificity in external validation:{np.mean(bootstrap_stats_class5['specificity']):.4f}", f"subclass5- Std-specificity in external validation: {np.std(bootstrap_stats_class5['specificity']):.4f}")
print(f"subclass5- Ave-sensitivity in external validation: {np.mean(bootstrap_stats_class5['recall']):.4f}", f"subclass5- Std-sensitivity in external validation:{np.std(bootstrap_stats_class5['recall']):.4f}")
print(f"subclass5- Ave-AUROC in external validation:{np.mean(bootstrap_stats_class5['auc']):.4f}", f"subclass5- Std-AUROC in external validation: {np.std(bootstrap_stats_class5['auc']):.4f}")
print(f"subclass5- Ave-precision in external validation: {np.mean(bootstrap_stats_class5['precision']):.4f}", f"subclass5- Std-precision in external validation: {np.std(bootstrap_stats_class5['precision']):.4f}")
print(f"subclass5- Ave-accyracy in external validation: {np.mean(bootstrap_stats_class5['accuracy']):.4f}", f"subclass5- Std-accuracy in external validation: {np.std(bootstrap_stats_class5['accuracy']):.4f}")
print(f"subclass5- Ave-F1 in external validation: {np.mean(bootstrap_stats_class5['F1']):.4f}", f"subclass5- Std-F1 in external validation: {np.std(bootstrap_stats_class5['F1']):.4f}")
print(f"subclass5- Ave-AUPRC in external validation: {np.mean(bootstrap_stats_class5['prc']):.4f}", f"subclass5- Std-AUPRC in external validation: {np.std(bootstrap_stats_class5['prc']):.4f}")

print(f"subclass6- Ave-specificity in external validation: {np.mean(bootstrap_stats_class6['specificity']):.4f}", f"subclass6- Std-specificity in external validation: {np.std(bootstrap_stats_class6['specificity']):.4f}")
print(f"subclass6- Ave-sensitivity in external validation: {np.mean(bootstrap_stats_class6['recall']):.4f}", f"subclass6- Std-sensitivity in external validation: {np.std(bootstrap_stats_class6['recall']):.4f}")
print(f"subclass6- Ave-AUROC in external validation: {np.mean(bootstrap_stats_class6['auc']):.4f}", f"subclass6- Std-AUROC in external validation: {np.std(bootstrap_stats_class6['auc']):.4f}")
print(f"subclass6- Ave-precision in external validation: {np.mean(bootstrap_stats_class6['precision']):.4f}", f"subclass6- Std-precision in external validation: {np.std(bootstrap_stats_class6['precision']):.4f}")
print(f"subclass6- Ave-accyracy in external validation: {np.mean(bootstrap_stats_class6['accuracy']):.4f}", f"subclass6- Std-accuracy in external validation: {np.std(bootstrap_stats_class6['accuracy']):.4f}")
print(f"subclass6- Ave-F1 in external validation: {np.mean(bootstrap_stats_class6['F1']):.4f}", f"subclass6- Std-F1 in external validation: {np.std(bootstrap_stats_class6['F1']):.4f}")
print(f"subclass6- Ave-AUPRC in external validation: {np.mean(bootstrap_stats_class6['prc']):.4f}", f"subclass6- Std-AUPRC in external validation: {np.std(bootstrap_stats_class6['prc']):.4f}")

print(f"subclass1- Ave-specificity in internal validation: {np.mean(bootstrap_stat_class1['specificity']):.4f}", f"subclass1- Std-specificity in internal validation: {np.std(bootstrap_stat_class1['specificity']):.4f}")
print(f"subclass1- Ave-sensitivity in internal validation: {np.mean(bootstrap_stat_class1['recall']):.4f}", f"subclass1- Std-sensitivity in internal validation: {np.std(bootstrap_stat_class1['recall']):.4f}")
print(f"subclass1- Ave-AUROC in internal validation: {np.mean(bootstrap_stat_class1['auc']):.4f}", f"subclass1- Std-AUROC in internal validation: {np.std(bootstrap_stat_class1['auc']):.4f}")
print(f"subclass1- Ave-precision in internal validation: {np.mean(bootstrap_stat_class1['precision']):.4f}", f"subclass1- Std-precision in internal validation: {np.std(bootstrap_stat_class1['precision']):.4f}")
print(f"subclass1- Ave-accyracy in internal validation: {np.mean(bootstrap_stat_class1['accuracy']):.4f}", f"subclass1- Std-accuracy in internal validation: {np.std(bootstrap_stat_class1['accuracy']):.4f}")
print(f"subclass1- Ave-F1 in internal validation: {np.mean(bootstrap_stat_class1['F1']):.4f}", f"subclass1- Std-F1 in internal validation: {np.std(bootstrap_stat_class1['F1']):.4f}")
print(f"subclass1- Ave-AUPRC in internal validation: {np.mean(bootstrap_stat_class1['prc']):.4f}", f"subclass1- Std-AUPRC in internal validation: {np.std(bootstrap_stat_class1['prc']):.4f}")

print(f"subclass2- Ave-specificity in internal validation: {np.mean(bootstrap_stat_class2['specificity']):.4f}", f"subclass2- Std-specificity in internal validation: {np.std(bootstrap_stat_class2['specificity']):.4f}")
print(f"subclass2- Ave-sensitivity in internal validation: {np.mean(bootstrap_stat_class2['recall']):.4f}", f"subclass2- Std-sensitivity in internal validation: {np.std(bootstrap_stat_class2['recall']):.4f}")
print(f"subclass2- Ave-AUROC in internal validation: {np.mean(bootstrap_stat_class2['auc']):.4f}", f"subclass2- Std-AUROC in internal validation: {np.std(bootstrap_stat_class2['auc']):.4f}")
print(f"subclass2- Ave-precision in internal validation: {np.mean(bootstrap_stat_class2['precision']):.4f}", f"subclass2- Std-precision in internal validation: {np.std(bootstrap_stat_class2['precision']):.4f}")
print(f"subclass2- Ave-accyracy in internal validation: {np.mean(bootstrap_stat_class2['accuracy']):.4f}", f"subclass2- Std-accuracy in internal validation: {np.std(bootstrap_stat_class2['accuracy']):.4f}")
print(f"subclass2- Ave-F1 in internal validation: {np.mean(bootstrap_stat_class2['F1']):.4f}", f"subclass2- Std-F1 in internal validation: {np.std(bootstrap_stat_class2['F1']):.4f}")
print(f"subclass2- Ave-AUPRC in internal cross-validation: {np.mean(bootstrap_stat_class2['prc']):.4f}", f"subclass2- Std-AUPRC in internal validation: {np.std(bootstrap_stat_class2['prc']):.4f}")

print(f"subclass3- Ave-specificity in internal validation: {np.mean(bootstrap_stat_class3['specificity']):.4f}", f"subclass3- Std-specificity in internal validation: {np.std(bootstrap_stat_class3['specificity']):.4f}")
print(f"subclass3- Ave-sensitivity in internal validation: {np.mean(bootstrap_stat_class3['recall']):.4f}", f"subclass3- Std-sensitivity in internal validation: {np.std(bootstrap_stat_class3['recall']):.4f}")
print(f"subclass3- Ave-AUROC in internal validation: {np.mean(bootstrap_stat_class3['auc']):.4f}", f"subclass3- Std-AUROC in internal validation: {np.std(bootstrap_stat_class3['auc']):.4f}")
print(f"subclass3- Ave-precision in internal validation: {np.mean(bootstrap_stat_class3['precision']):.4f}", f"subclass3- Std-precision in internal validation: {np.std(bootstrap_stat_class3['precision']):.4f}")
print(f"subclass3- Ave-accyracy in internal validation: {np.mean(bootstrap_stat_class3['accuracy']):.4f}", f"subclass3- Std-accuracy in internal validation: {np.std(bootstrap_stat_class3['accuracy']):.4f}")
print(f"subclass3- Ave-F1 in internal validation: {np.mean(bootstrap_stat_class3['F1']):.4f}", f"subclass3- Std-F1 in internal validation: {np.std(bootstrap_stat_class3['F1']):.4f}")
print(f"subclass3- Ave-AUPRC in internal validation: {np.mean(bootstrap_stat_class3['prc']):.4f}", f"subclass3 Std-AUPRC in internal validation: {np.std(bootstrap_stat_class3['prc']):.4f}")

print(f"subclass4- Ave-specificity in internal validation: {np.mean(bootstrap_stat_class4['specificity']):.4f}", f"subclass4- Std-specificity in internal validation: {np.std(bootstrap_stat_class4['specificity']):.4f}")
print(f"subclass4- Ave-sensitivity in internal validation: {np.mean(bootstrap_stat_class4['recall']):.4f}", f"subclass4- Std-sensitivity in internal validation: {np.std(bootstrap_stat_class4['recall']):.4f}")
print(f"subclass4- Ave-AUROC in internal validation:{np.mean(bootstrap_stat_class4['auc']):.4f}", f"subclass4- Std-AUROC in internal validation: {np.std(bootstrap_stat_class4['auc']):.4f}")
print(f"subclass4- Ave-precision in internal validation: {np.mean(bootstrap_stat_class4['precision']):.4f}", f"subclass4- Std-precision in internal validation: {np.std(bootstrap_stat_class4['precision']):.4f}")
print(f"subclass4- Ave-accyracy in internal validation: {np.mean(bootstrap_stat_class4['accuracy']):.4f}", f"subclass4- Std-accuracy in internal validation: {np.std(bootstrap_stat_class4['accuracy']):.4f}")
print(f"subclass4- Ave-F1 in internal validation:{np.mean(bootstrap_stat_class4['F1']):.4f}", f"subclass4- Std-F1 in internal validation: {np.std(bootstrap_stat_class4['F1']):.4f}")
print(f"subclass4- Ave-AUPRC in internal validation: {np.mean(bootstrap_stat_class4['prc']):.4f}", f"subclass4- Std-AUPRC in internal validation: {np.std(bootstrap_stat_class4['prc']):.4f}")

print(f"subclass5- Ave-specificity in internal validation: {np.mean(bootstrap_stat_class5['specificity']):.4f}", f"subclass5- Std-specificity in internal validation: {np.std(bootstrap_stat_class5['specificity']):.4f}")
print(f"subclass5- Ave-sensitivity in internal validation: {np.mean(bootstrap_stat_class5['recall']):.4f}", f"subclass5- Std-sensitivity in internal validation: {np.std(bootstrap_stat_class5['recall']):.4f}")
print(f"subclass5- Ave-AUROC in internal validation: {np.mean(bootstrap_stat_class5['auc']):.4f}", f"subclass5- Std-AUROC in internal validation: {np.std(bootstrap_stat_class5['auc']):.4f}")
print(f"subclass5- Ave-precision in internal validation: {np.mean(bootstrap_stat_class5['precision']):.4f}", f"subclass5- Std-precision in internal validation: {np.std(bootstrap_stat_class5['precision']):.4f}")
print(f"subclass5- Ave-accyracy in internal validation: {np.mean(bootstrap_stat_class5['accuracy']):.4f}", f"subclass5- Std-accuracy in internal validation: {np.std(bootstrap_stat_class5['accuracy']):.4f}")
print(f"subclass5- Ave-F1 in internal validation: {np.mean(bootstrap_stat_class5['F1']):.4f}", f"subclass5- Std-F1 in internal validation: {np.std(bootstrap_stat_class5['F1']):.4f}")
print(f"subclass5- Ave-AUPRC in internal alidation: {np.mean(bootstrap_stat_class5['prc']):.4f}", f"subclass5- Std-AUPRC in internal validation: {np.std(bootstrap_stat_class5['prc']):.4f}")

print(f"subclass6- Ave-specificity in internal validation: {np.mean(bootstrap_stat_class6['specificity']):.4f}", f"subclass6- Std-specificity in internal validation: {np.std(bootstrap_stat_class6['specificity']):.4f}")
print(f"subclass6- Ave-sensitivity in internal validation: {np.mean(bootstrap_stat_class6['recall']):.4f}", f"subclass6- Std-sensitivity in internal validation: {np.std(bootstrap_stat_class6['recall']):.4f}")
print(f"subclass6- Ave-AUROC in internal validation:{np.mean(bootstrap_stat_class6['auc']):.4f}", f"subclass6- Std-AUROC in internal validation: {np.std(bootstrap_stat_class6['auc']):.4f}")
print(f"subclass6- Ave-precision in internal validation: {np.mean(bootstrap_stat_class6['precision']):.4f}", f"subclass6- Std-precision in internal validation: {np.std(bootstrap_stat_class6['precision']):.4f}")
print(f"subclass6- Ave-accyracy in internal validation:: {np.mean(bootstrap_stat_class6['accuracy']):.4f}", f"subclass6- Std-accuracy in internal validation: {np.std(bootstrap_stat_class6['accuracy']):.4f}")
print(f"subclass6- Ave-F1 in internal validation: {np.mean(bootstrap_stat_class6['F1']):.4f}", f"subclass6- Std-F1 in internal validation: {np.std(bootstrap_stat_class6['F1']):.4f}")
print(f"subclass6- Ave-AUPRC in internal validation:{np.mean(bootstrap_stat_class6['prc']):.4f}", f"subclass6- Std-AUPRC in internal validation: {np.std(bootstrap_stat_class6['prc']):.4f}")